## Definition

### Problem Statement  

The goal is to forecast the demand for bikes in dependency of weather conditions like outside temperature and calendric informations e.g. holidays. These information and the demand structure is provided in a set with two years of daily historic data.  
The demand is given as the total daily demand and as a split for registered users and casual users. To increase the quality of the prediction registered user demand and casual user demand will be predicted separately in step two.  
To make predictions machine learning is used to train regressors. Scikit-Learn recommends a support vector regressor (SVR) for this kind of problem and data amount. In addition a deep neuronal network (DNN) regressor is trained for comparison. To find the hyper-parameters for these regressors grid search and randomized search are utilized. Due to the small dataset cross validation is applied.    

> http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR  
> https://github.com/tensorflow/skflow/blob/master/g3doc/api_docs/python/estimators.md  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

In [1]:
# Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar

from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
from math import sqrt

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Analysis

In [ ]:
# Fetching Dataset

bike_data = pd.read_csv("day.csv", header=0)

print("Data read successfully!")

In [ ]:
bike_data.head()

### Data Exploration

In [ ]:
# Extracting

feature_cols = bike_data.columns[:-3]  # all columns but last are features
target_col = bike_data.columns[-1]  # last column is the target

print ("Feature column(s):\n{}\n".format(feature_cols))
print ("Target column:\n{}".format(target_col))

In [ ]:
#### Function to Calculate Profit

In [2]:
def profit(y,y_cap):
    return 3 * np.minimum(y[::1], y_cap[::1]) - 2 * y_cap[::1]
    

#### Function to Convert from percentage to Actual Prediction

In [3]:
def convertToPrediction(data,percentage_predictions):
      return np.around(data + (np.multiply(data, percentage_predictions)/100))

### Base Model

#### For the base model the demand for today is the previous days demand. 

In [ ]:
y_actual = bike_data[target_col][365:731]  # corresponding targets
y_actual = y_actual.reset_index(drop = True)

In [ ]:
y_staged = y_actual.copy()

In [ ]:
data = []
data.insert(0, bike_data[target_col][364])
data.insert(0, bike_data[target_col][363])

In [ ]:
y_predicted_df = pd.concat([pd.DataFrame(data), y_staged], ignore_index=True)

In [ ]:
y_predicted_df.drop(y_predicted_df.tail(2).index,inplace=True)

In [ ]:
y_predicted = y_predicted_df[0]


##### Calculate Base Model Profit

In [ ]:
print(profit(y_actual,y_predicted).sum())

### Benchmark

In [ ]:
X_raw_train = pd.read_csv("train.csv", header=0)
X_raw_test  = pd.read_csv("test.csv", header=0)

In [ ]:
cols = ["temp","hum", "windspeed" ,"cnt_normal","week_moving_avg_normal","season_1","season_2","season_3","season_4","mnth_1","mnth_2","mnth_3","mnth_4","mnth_5","mnth_6","mnth_7","mnth_8","mnth_9","mnth_10","mnth_11","mnth_12","holiday_1","holiday_2","weekday_1","weekday_2","weekday_3","weekday_4","weekday_5","weekday_6","weekday_7","workingday_1","workingday_2","weathersit_1","weathersit_2","weathersit_3"]

In [ ]:
cols = ["atemp","hum", "windspeed" ,"cnt_normal","week_moving_avg_normal","season_1","season_2","season_3","season_4","mnth_1","mnth_2","mnth_3","mnth_4","mnth_5","mnth_6","mnth_7","mnth_8","mnth_9","mnth_10","mnth_11","mnth_12","holiday_1","holiday_2","weekday_1","weekday_2","weekday_3","weekday_4","weekday_5","weekday_6","weekday_7","workingday_1","workingday_2","weathersit_1","weathersit_2","weathersit_3"]

In [ ]:
X_train = X_raw_train[cols].values.tolist()
y_train_df = X_raw_train[['target']]
y_train = y_train_df['target'].tolist()

In [ ]:
X_test = X_raw_test[cols].values.tolist()
y_test_df = X_raw_test[['target']]
y_test = y_test_df['target'].tolist()

#### Alternate dataset with percentage change

In [79]:
data = pd.read_csv("processed_Data.csv", header=0)
data['instant'] = data['instant'] % 30
X_raw_train = data[0:359]
X_raw_test  = data[359:]

In [80]:
cols =[
       "season__1","season__2","season__3","season__4","season__5",
       "weathersit__1","weathersit__2","weathersit__3","weathersit__4","weathersit__5",
        "cnt__1",
        "atemp","hum","windspeed",
        "mnth","instant","holiday","weekday","workingday",
        "moving_avg_weekly_cnt"]     

In [81]:
X_train = X_raw_train[cols].values.tolist()
y_train_df = X_raw_train[['demand_pc_inc']]
y_train = y_train_df['demand_pc_inc'].tolist()

In [82]:
X_test = X_raw_test[cols].values.tolist()
y_test_df = X_raw_test[['demand_pc_inc']]
y_test = y_test_df['demand_pc_inc'].tolist()

In [83]:
data_cnt = data['cnt']

In [84]:
actual_predictions = data_cnt[359:].values

In [85]:
y_for_calculations = data_cnt[357:723].values

### Algorithms and Techniques

In [109]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

In [110]:
#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error # for regression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score  # for classification
 

In [201]:
models=[RandomForestRegressor(),GradientBoostingRegressor(),AdaBoostRegressor(),BaggingRegressor(),SVR(),KNeighborsRegressor()]
model_names=['RandomForestRegressor','GradientBoostingRegressor','AdaBoostRegressor','BaggingRegressor','SVR','KNeighborsRegressor']
rmsle=[]
d={}
for model in range (len(models)):
    clf=models[model]
    print(model_names[model])
    clf.fit(X_train,y_train)
    test_pred=clf.predict(X_test)
    model_predictions = convertToPrediction(y_for_calculations,test_pred)
    print(profit(actual_predictions,model_predictions).sum())

RandomForestRegressor
1571240.0
GradientBoostingRegressor
1585123.0
AdaBoostRegressor
1500697.0
BaggingRegressor
1588207.0
SVR
1438256.0
KNeighborsRegressor
1445710.0


## Methodology

In [223]:
#NOW LET'S Dig deeper into each of these ...

### Implementation

The regressors are trained using randomized search and cross-validation to identify the area of the best parameters. Then a grid search is used to tune parameter values of the regressor functions.

> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html  
> http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html

##### Random Forest Regressor

In [112]:

#for random forest regresion.
no_of_test=[500]
params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30]}
clf_rf=GridSearchCV(estimator=RandomForestRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_rf.fit(X_train,y_train)
pred_rf=clf_rf.predict(X_test)
model_predictions_rf = convertToPrediction(y_for_calculations,pred_rf)
print(profit(actual_predictions,model_predictions_rf).sum())

1547591.0


In [126]:
print("Best params: ", clf_rf.best_params_)

Best params:  {'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 500, 'n_jobs': -1}


In [165]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = RandomForestRegressor()
estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator.fit(X_train, y_train)
pred_rf = estimator.predict(X_test)
model_predictions_rf = convertToPrediction(y_for_calculations,pred_rf)
print(profit(actual_predictions,model_predictions_rf).sum())

1591630.0


#### Gradient Boosting Regressor

In [135]:
#for Gradient Boosting regresion.
no_of_estimators=[100,200,300,400,500]
params_dict={'n_estimators':no_of_estimators,'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
             'max_features':["auto",'sqrt','log2'],'max_depth':[10,20,30,40,50]}
clf_gbr=GridSearchCV(estimator=GradientBoostingRegressor(),param_grid=params_dict,scoring='neg_mean_squared_error')
clf_gbr.fit(X_train,y_train)
pred=clf_gbr.predict(X_test)
model_predictions = convertToPrediction(y_for_calculations,pred)
print(profit(actual_predictions,model_predictions).sum())

1452701.0


In [136]:
print("Best params: ", clf_gbr.best_params_)

Best params:  {'learning_rate': 0.2, 'max_depth': 40, 'max_features': 'log2', 'n_estimators': 100}


In [137]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=500,max_features= 'log2', learning_rate=0.1, max_depth = 10)

estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', gbr)
    ]
)
estimator.fit(X_train, y_train)
pred_gbr = estimator.predict(X_test)
model_predictions_gbr = convertToPrediction(y_for_calculations,pred_gbr)
print(profit(actual_predictions,model_predictions_gbr).sum())

1556243.0


#### ADA Boost Regressor

In [192]:
### ADA Boost Regressor

from sklearn.ensemble import AdaBoostRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = AdaBoostRegressor()
estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator.fit(X_train, y_train)
pred_ada = estimator.predict(X_test)
model_predictions_ada = convertToPrediction(y_for_calculations,pred_ada)
print(profit(actual_predictions,model_predictions_ada).sum())

1512214.0


#### Bagging Regressor

In [195]:
### Bagging Regressor

from sklearn.ensemble import BaggingRegressor
from sklearn import pipeline,metrics,grid_search

#regressor = RandomForestRegressor(random_state = 0, max_depth = 30, n_estimators = 500, max_features = 'log2')
regressor = BaggingRegressor()
estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator.fit(X_train, y_train)
pred_bagging = estimator.predict(X_test)
model_predictions_bagging = convertToPrediction(y_for_calculations,pred_bagging)
print(profit(actual_predictions,model_predictions_bagging).sum())


1551922.0


#### KNN Regressor

In [219]:
### KNN Regressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import pipeline,metrics,grid_search


regressor = KNeighborsRegressor()
estimator = pipeline.Pipeline(steps = [       
    ('model_fitting', regressor)
    ]
)
estimator.fit(X_train, y_train)
pred_knn = estimator.predict(X_test)
model_predictions_knn = convertToPrediction(y_for_calculations,pred_knn)
print(profit(actual_predictions,model_predictions_knn).sum())




1445710.0


#### SVM Regressor

In [86]:
# Training SVR
svr = SVR()
svr.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [215]:
# Validation SVR

pred_svr = svr.predict(X_test)
score_svr = r2_score(y_test, pred_svr)
rmse_svr = sqrt(mean_squared_error(y_test, pred_svr))

print("Score SVR: %f" % score_svr)
print("RMSE SVR: %f" % rmse_svr)

Score SVR: -0.003305
RMSE SVR: 1318.397271


In [ ]:
# Tuning SVR with GridSearch

tuned_parameters = [{'C': [1000, 3000, 10000], 
                     'kernel': ['linear', 'rbf']}
                   ]

#svr_tuned = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'mean_squared_error') #default 3-fold cross-validation, score method of the estimator
svr_tuned_GS = GridSearchCV(SVR (C=1), param_grid = tuned_parameters, scoring = 'r2', n_jobs=-1) #default 3-fold cross-validation, score method of the estimator

svr_tuned_GS.fit(X_train, y_train)

print (svr_tuned_GS)
print ('\n' "Best parameter from grid search: " + str(svr_tuned_GS.best_params_) +'\n')

In [ ]:
svr_tuned_pred_GS = svr_tuned_GS.predict(X_test)

score_svr_tuned_GS = r2_score(y_test, svr_tuned_pred_GS)
rmse_svr_tuned_GS = sqrt(mean_squared_error(y_test, svr_tuned_pred_GS))

print("SVR Results\n")

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)

In [ ]:
svr_tuned_pred_GS


##Profit Calculation for pct approach
model_predictions = convertToPrediction(y_for_calculations,svr_tuned_pred_GS)
print(profit(actual_predictions,model_predictions).sum())

#Profit is just 1.26million!!

In [ ]:
# SVR tuned with RandomizesSearch
# may take a while!

# Parameters
param_dist = {  'C': sp_uniform (1000, 10000), 
                'kernel': ['rbf']
             }

n_iter_search = 1

# MSE optimized
#SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'mean_squared_error', n_iter=n_iter_search)

# R^2 optimized
SVR_tuned_RS = RandomizedSearchCV(SVR (C=1), param_distributions = param_dist, scoring = 'r2', n_iter=n_iter_search)

# Fit
SVR_tuned_RS.fit(X_train, y_train)

# Best score and corresponding parameters.
print('best CV score from grid search: {0:f}'.format(SVR_tuned_RS.best_score_))
print('corresponding parameters: {}'.format(SVR_tuned_RS.best_params_))

# Predict and score
predict = SVR_tuned_RS.predict(X_test)

score_svr_tuned_RS = r2_score(y_test, predict)
rmse_svr_tuned_RS = sqrt(mean_squared_error(y_test, predict))

In [ ]:
print('SVR Results\n')

print("Score SVR: %f" % score_svr)
print("Score SVR tuned GS: %f" % score_svr_tuned_GS)
print("Score SVR tuned RS: %f" % score_svr_tuned_RS)

print("\nRMSE SVR: %f" % rmse_svr)
print("RMSE SVR tuned GS: %f" % rmse_svr_tuned_GS)
print("RMSE SVR tuned RS: %f" % rmse_svr_tuned_RS)

In [ ]:
##Profit Calculation for pct approach
model_predictions = convertToPrediction(y_for_calculations,predict)
print(profit(actual_predictions,model_predictions).sum())

### DNN Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
import logging
from concurrent.futures import ThreadPoolExecutor, wait
from time import time
from typing import List

In [ ]:
bike_model = MLPRegressor(hidden_layer_sizes=(5,),
                                       activation='relu',
                                       solver='adam',
                                       learning_rate='adaptive',
                                       max_iter=15000,
                                       learning_rate_init=0.01,
                                       alpha=0.01)

In [ ]:
start_time = int(time() * 1000)
bike_model.fit(X_train, y_train)
end_time = int(time() * 1000)
logging.debug('Finished training universal model')
logging.debug('Training took {} ms'.format(end_time - start_time)) 

In [ ]:
pred_dnn = bike_model.predict(X_test)

In [ ]:
model_predictions_dnn = convertToPrediction(y_for_calculations,pred_dnn)

In [ ]:
print(profit(actual_predictions,model_predictions_dnn).sum())

#### Ensembles

In [224]:
# Averageing best predictions from different regressors

In [226]:
w = [1, 1, 1, 1, 1,1,1]  # weights
pred_agg = np.c_[pred_rf,pred_gbr,pred_ada,pred_bagging,pred_knn,pred_svr,pred_dnn ]
pred_avr = np.average(pred_agg, axis=1, weights=w)
model_predictions_avr = convertToPrediction(y_for_calculations,pred_avr)
print(profit(actual_predictions,model_predictions_avr).sum())

1547627.0
